In [ ]:
import os
import matplotlib.pyplot    as plt
import numpy                as np
import matplotlib.patches   as mpatches
import plotly.graph_objects as go
from   scipy.stats      import skew, kurtosis, mode
from   scipy.spatial    import KDTree
from   sklearn.cluster  import DBSCAN
from   scipy.ndimage    import convolve
from   collections      import Counter

In [ ]:
def get_vectors(data1, data2, x_vals, y_span=20):
    vectors = {}
    for x in x_vals:
        distribution = get_distribution(data1,data2,x, y_span) 
        properties = get_props(distribution)
        if not np.any(np.isnan(properties)):
            vectors[x] = properties
    return vectors

def get_vectors_convolved(data1, data2, x_vals, y_span=20):
    vectors = {}
    for x in x_vals:
        distribution = get_distribution(data1,data2,x, y_span)
        if distribution.size > 0: 
            properties = get_props_convolved(distribution)
        else:
            properties = [0,0,0,0,0,0,0]
        if not np.any(np.isnan(properties)):
            vectors[x] = properties
    return vectors

def get_vectors_combined(tree, og, convolved, x_vals, y_span=20):
    vectors = {}
    for x in x_vals:
        conv_dist = get_distribution(convolved,tree,x, y_span)
        dist = get_distribution(og,tree,x,y_span)
        if conv_dist.size > 0: 
            conv_props = get_props_convolved(conv_dist)
        else:
            conv_props = [0,0,0,0,0,0,0]
        properties = np.hstack((conv_props, get_props(dist)))
        if not (np.any(np.isnan(properties))):
                vectors[x] = properties
    return vectors

def get_distribution(data1, data2, point, y_span):

    indices = data2.query_ball_point((point,0), y_span)

    result_points = data1[indices]

    filtered_points = result_points[
        (result_points[:, 0] == point) &
        (np.abs(result_points[:, 1]) <= y_span) & 
        (result_points[:,2] != 0)
    ]

    return filtered_points[:, 2]


def get_props_convolved(distribution):
    properties = []

    mean = np.sqrt(np.mean(distribution))
    max = np.sqrt(np.max(distribution))

    variance =  np.var(distribution)
    skw = skew(distribution)
    kurt = kurtosis(distribution)

    
    properties.append(variance)
    properties.append(skw)
    properties.append(kurt)
    

    norm = np.linalg.norm(properties)
    
    if norm > 0:
        properties = [x / norm for x in properties]
    

    properties.append(mean)
    properties.append(max)

    z = properties[0]
    y = properties[1]
    x = properties[2]

    properties.append((np.atan2(y,x) + (2 * np.pi)) % (2 * np.pi))
    properties.append(np.atan(z)/(np.pi/2))


    return properties

def get_props(distribution):
    properties = []

    variance =  np.var(distribution)
    skw = skew(distribution)
    kurt = kurtosis(distribution)

    
    properties.append(variance)
    properties.append(skw)
    properties.append(kurt)

    norm = np.linalg.norm(properties)
    
    if norm > 0:
        properties = [x / norm for x in properties]

    z = properties[0]

    properties.append(np.acos(z)/(np.pi/2))

    
    return properties



In [ ]:
def trim_cloud(data,start,stop,span = 22):
        data = data[
            (data[:,0] >= stop) & 
            (data[:,0] <= start) 
            ]
        data = data[(data[:,1] >= -span) & (data[:,1] <= span)]
        return data


def get_point_cloud(file_path):

    point_cloud = []

    with open(os.path.join(file_path, '.component_parameters.txt')) as file:
            lines = file.readlines()
            for line in lines:
                if "XRAY_DPP[Acquisition]#0.Y.Start:" in line:
                    y_offset = (float)(line.split("XRAY_DPP[Acquisition]#0.Y.Start:")[1].strip())
                elif "XRAY_DPP[Acquisition]#0.X.Start:" in line:
                    x_start = (float)(line.split("XRAY_DPP[Acquisition]#0.X.Start:")[1].strip())
                elif "XRAY_DPP[Acquisition]#0.X.Stop:" in line:
                    x_stop = (float)(line.split("XRAY_DPP[Acquisition]#0.X.Stop:")[1].strip())


    if os.path.isdir(file_path):
        lidar_files = [fn for fn in os.listdir(
            file_path) if fn.endswith('.bpc')]
        if any(lidar_files):
            lidar_filename = file_path + os.sep + lidar_files[0]

    data = np.fromfile(lidar_filename, dtype=np.float32)
    point_cloud = data.reshape(-1, 3) 

    ff = ~np.isnan(point_cloud).any(axis=1)
    point_cloud = point_cloud[ff, ...]


    print(f"{file_path} is loaded. \n# of point {point_cloud.shape[0]}")

    matrix_file = (os.path.join(file_path, ".XRAY_DPP_001.lidar2xrf"))
    with open(matrix_file, 'r') as file:
        lines = file.readlines()

    transformation_matrix = np.array([list(map(float, line.strip().split(","))) for line in lines])

    num_points = point_cloud.shape[0]


    homogeneous_points = np.hstack((point_cloud, np.ones((num_points, 1))))
    transformed_points = homogeneous_points @ transformation_matrix.T
    point_cloud = transformed_points[:, :3]


    point_cloud = trim_cloud(point_cloud,x_start,x_stop)


    return point_cloud, x_start, x_stop


In [ ]:
def convolve_data(data,kernels):
    x_values = np.unique(data[:,0])
    y_values = np.unique(data[:,1])
    array = np.zeros((len(y_values), len(x_values)))
    map = {(row[0],row[1]): (index, row[2]) for index,row in enumerate(data)}

    for x in range (len(x_values)):
        for y in range (len(y_values)):
            x_val = x_values[x]
            y_val = y_values[y]
            z = map.get((x_val,y_val))
            if z is not None:
                array[y][x] = z[1]

    result_array = np.zeros((len(y_values), len(x_values)))
    
    for kernel in kernels:
        result_array += np.abs(convolve(array,kernel))

    return_data = data.copy()

    for x in range (len(x_values)):
        for y in range (len(y_values)):
            x_val = x_values[x]
            y_val = y_values[y]
            index = map.get((x_val,y_val))
            if index is not None:
                return_data[index[0],2] = result_array[y][x]
    return return_data

def canny(data):
    sobel_x = np.array([
        [-1, 0, 1],
        [-2, 0, 2],
        [-1, 0, 1]
    ])
    sobel_y = np.array([
        [ 1, 2, 1],
        [ 0, 0, 0],
        [-1,-2,-1]
    ])
    angs = np.array([0, np.pi/4, np.pi/2, 3*np.pi/4])

    x_values = np.unique(data[:,0])
    y_values = np.unique(data[:,1])
    array = np.zeros((len(y_values), len(x_values)))
    map = {(row[0],row[1]): (index, row[2]) for index,row in enumerate(data)}

    for x in range (len(x_values)):
        for y in range (len(y_values)):
            x_val = x_values[x]
            y_val = y_values[y]
            z = map.get((x_val,y_val))
            if z is not None:
                array[y][x] = z[1]

    y_grad = np.zeros((len(y_values), len(x_values)))
    x_grad = np.zeros((len(y_values), len(x_values)))
    NMS_array = np.zeros(array.shape, dtype=object)

    
    y_grad = convolve(array,sobel_y)
    x_grad = convolve(array,sobel_x)


    for x in range (len(x_values)):
        for y in range (len(y_values)):
            Gx = x_grad[y][x]
            Gy = y_grad[y][x]

            Gtheta  = np.atan2(Gy,Gx) % np.pi
            
            theta = np.argmin(np.abs(Gtheta - angs))

            NMS_array[y][x] = tuple((np.sqrt(Gx**2 + Gy**2),theta))

    return_data = data.copy()


    for x in range (1,len(x_values) - 1):
        for y in range (1,len(y_values) - 1):
            index = map.get((x_values[x],y_values[y]))
            if index is not None:
                i = index[0]
                mag = (NMS_array[y][x])[0]
                ang = (NMS_array[y][x])[1]
                
                if (ang == 0):
                    n1 = (NMS_array[y][x + 1])[0]
                    n2 = (NMS_array[y][x - 1])[0]
                if (ang == 1):
                    n1 = (NMS_array[y - 1][x - 1])[0]
                    n2 = (NMS_array[y + 1][x + 1])[0]
                if (ang == 2):
                    n1 = (NMS_array[y + 1][x])[0]
                    n2 = (NMS_array[y - 1][x])[0]
                if (ang == 3):
                    n1 = (NMS_array[y - 1][x + 1])[0]
                    n2 = (NMS_array[y + 1][x - 1])[0]
                if mag >= n1 and mag >= n2:
                    return_data[i,2] = mag
                else:
                    return_data[i,2] = 0


    return_data[:, 2] = np.where(return_data[:, 2] >= 2, 1,0)

    #return_data[:, 2] = np.clip(return_data[:, 2], 0.1, 250)
    
    return return_data 


def get_hessian_dictionary(data):
    hessian_dict = {}

    kernel_xx = np.array([
    [ 1, -2,  1],
    [ 2, -4,  2],
    [ 1, -2,  1]
    ])
    kernel_yy = np.array([
    [ 1,  2,  1],
    [-2, -4, -2],
    [ 1,  2,  1]
    ])
    kernel_xy = np.array([
    [ 0,  1,  0],
    [ 1, -4,  1],
    [ 0,  1,  0]
    ])


    x_values = np.unique(data[:,0])
    y_values = np.unique(data[:,1])
    array = np.zeros((len(y_values), len(x_values)))
    map = {(row[0],row[1]): (index, row[2]) for index,row in enumerate(data)}

    for x in range (len(x_values)):
        for y in range (len(y_values)):
            x_val = x_values[x]
            y_val = y_values[y]
            z = map.get((x_val,y_val))
            if z is not None:
                array[y][x] = z[1]

    dxx = convolve(array,kernel_xx)
    dyy = convolve(array,kernel_yy)
    dxy = convolve(array,kernel_xy)


    for x in range (len(x_values)):
        for y in range (len(y_values)):
            x_val = x_values[x]
            y_val = y_values[y]
            index = map.get((x_val,y_val))
            if index is not None:
                point_dxx = dxx[y][x]
                point_dyy = dyy[y][x]
                point_dxy = dxy[y][x]

                hessian = np.array([[point_dxx, point_dxy], [point_dxy, point_dyy]])
                eigens = np.linalg.eig(hessian)
                hessian_dict[x_val,y_val] = eigens
    
    return hessian_dict

In [ ]:
#Get the paths of each part
def find_parts(directory_path, str = "AdaptiveZ_10mm"):
    parts_paths = []

    for root, dirs, files in os.walk(directory_path):
        for folder in dirs:
            folder = os.path.join(root,folder)
            if (str in folder) and folder.endswith("_4"):
                parts_paths.append(folder)
    parts_paths.sort(reverse=True)
    return parts_paths


base_path = r"\\192.168.1.100\CoreScan3-2\Acquisitions\RnD\XRF\CH\Macassa_clearance"



paths = find_parts(base_path)

In [ ]:
#Build the point cloud
point_clouds = []

global x_offset
x_offset = 0

for path in paths:
    temp_cloud, x_start, x_stop = get_point_cloud(path)
    x_offset -= x_stop
    temp_cloud[:,0] += x_offset
    x_offset += x_start
    point_clouds.append(temp_cloud)
   
point_cloud = np.vstack(point_clouds)

print(f"total # of point {point_cloud.shape[0]}")

point_tree = KDTree(point_cloud[:,:2])

In [ ]:
#add to the point cloud
#point_clouds = []
#
#for path in paths:
#    temp_cloud, x_start, x_stop = get_point_cloud(path)
#    x_offset -= x_stop
#    temp_cloud[:,0] += x_offset
#    x_offset += x_start
#    point_clouds.append(temp_cloud)
#
#point_cloud_add = np.vstack(point_clouds)
#point_cloud = np.concatenate((point_cloud,point_cloud_add))
#
#print(f"total # of point {point_cloud.shape[0]}")
#
#point_tree = KDTree(point_cloud[:,:2])

In [ ]:
#create a convolved copy of the point_cloud

kernels_y = []
kernels_s = []
kernels_x = []

kernels_s.append(np.array([[1,2,1],[2,4,2],[1,2,1]]))
kernels_y.append(np.array([[-1],[-2],[0],[2],[1]]))
kernels_x.append(np.array([[-1,-2,0,2,1]]))


point_cloud_convolved = convolve_data(point_cloud,kernels_y)

point_cloud_edges = canny(convolve_data(point_cloud,kernels_s))


In [ ]:
#further define border edges 



In [ ]:
#create a point_cloud with edges removed 

edge_indices = np.where(point_cloud_edges[:,2] == 1)[0]

point_cloud_edgeless = point_cloud.copy()

for i in edge_indices:
    point_cloud_edgeless[i, 2] = np.nan

In [ ]:
#fig = plt.figure(figsize=(500,10), dpi = 400)
#
#
#gs = fig.add_gridspec(2, 1, hspace = 0.8)
#
#downsampled_indices = np.random.choice(point_cloud.shape[0], size=5000000, replace=False)
#x_downsampled = point_cloud_edgeless[:, 0]
#y_downsampled = point_cloud_edgeless[:, 1]
#z_downsampled = point_cloud_edgeless[:, 2]
#
#
#
#
#x_values = np.unique(point_cloud[:,0])
#
#ax_scatter = fig.add_subplot(gs[0,0])
#
#ax_scatter.scatter(
#x_downsampled,
#y_downsampled,
#c=z_downsampled,
#cmap='binary_r',  
#s = 0.25
#)
#
#
#
#
#
#ax_scatter.set_title('LIDAR data heatmap')
#ax_scatter.set_xlabel('X-axis')
#ax_scatter.set_ylabel('Y-axis')
#
#plt.show()
#


In [ ]:
#initial clustering with the original point cloud

# 0 = variance, 1 = skew, 2 = kurtosis, 3 = mean, 4 = polar


x_values = np.unique(point_cloud[:,0])

v_dict = get_vectors(point_cloud,point_tree,x_values, y_span = 10)
v_array = np.array(list(v_dict.values()))
print(v_array.shape)

In [ ]:
# 0 = variance, 1 = skew, 2 = kurtosis, 3 = polar
dimensions = [1,3] 

dbscan = DBSCAN(min_samples = 50, 
                metric = 'chebyshev', 
                eps = 0.054, 
                n_jobs = -1) 

labels1 = dbscan.fit_predict(v_array[:, dimensions])

defined_indices = np.where(np.logical_or(labels1 == 0, labels1 == 2))[0]
semi_defined_indices = np.where(labels1 == 1)[0]

core_type_dictionary = {x: labels1[i] for i,x in enumerate(v_dict.keys())}

fig = plt.figure(figsize=(10,4), dpi = 300)

colors = [f"C{label}"  if label >= 0 else (0,1,1,1)  for label in labels1]

gs = fig.add_gridspec(1, 1 )
ax = fig.add_subplot(gs[0,0])



x_vals = [v_dict[x][1] for x in v_dict.keys()]
y_vals = [v_dict[x][3] for x in v_dict.keys()]
ax.set_xlabel("skew")
ax.set_ylabel("Azimuthal angle")


ax.scatter(x_vals, y_vals, color=colors, s=1)


In [ ]:
fig = plt.figure(figsize=(150,15), dpi = 300)


gs = fig.add_gridspec(2, 1, hspace = 0.8)

downsampled_indices = np.random.choice(point_cloud.shape[0], size=1200000, replace=False)
x1_downsampled = point_cloud[downsampled_indices, 0]
y1_downsampled = point_cloud[downsampled_indices, 1]
z1_downsampled = point_cloud[downsampled_indices, 2]



downsampled_indices = np.where(point_cloud_edges[:,2] == 1)[0]

print(len(downsampled_indices))

x_downsampled = point_cloud_edges[downsampled_indices, 0]
y_downsampled = point_cloud_edges[downsampled_indices, 1]
z_downsampled = point_cloud_edges[downsampled_indices, 2]


x_values = np.unique(point_cloud[:,0])

ax_scatter = fig.add_subplot(gs[0,0])
#ax_scatter.scatter(
#x1_downsampled,
#y1_downsampled,
#c=z1_downsampled,
#cmap='binary_r',  
#s = 0.25
#)

ax_scatter.scatter(
x_downsampled,
y_downsampled,
c=z_downsampled,
cmap='binary_r',  
s = 0.25
)





ax_scatter.set_title('LIDAR data heatmap')
ax_scatter.set_xlabel('X-axis')
ax_scatter.set_ylabel('Y-axis')

ax_scatter.axhline(y=0.03, color='red', linestyle='-', label="Horizontal Line")
#bar_plot = fig.add_subplot(gs[1,0])
#
#
#legend_labels = np.unique(labels1)
#legend_colors = [f"C{label}"  if label >= 0 else (0,0,0,0) for label in legend_labels]
#legend_handles = [mpatches.Patch(color=color, label=label) for label, color in zip(legend_labels, legend_colors)]
#
#bar_plot.bar(x_values, height=1.5, width=1.5, color=colors, edgecolor="none")
#
#bar_plot.legend(handles=legend_handles, title="Core Type", bbox_to_anchor=(1,1), loc="upper left")
#
#bar_plot.set_title("Core Type by X-position")
#bar_plot.set_xlabel("X-Value")
#bar_plot.set_yticks([])
#bar_plot.set_xticks(list(range(0, 31001, 1000)))
#bar_plot.set_xticklabels(list(range(0, 31001, 1000)), rotation=70)

plt.show()

In [ ]:
#getting convolved vectors from undefined indices

x_values = [x for i, x in enumerate(v_dict.keys()) if i in semi_defined_indices]

#x_values = np.unique(point_cloud[:,0])


#v_dict_convolved = get_vectors_convolved(point_cloud_convolved, point_tree, x_values, y_span = 10)
v_dict_convolved = get_vectors_combined(point_tree,point_cloud,point_cloud_convolved,x_values, y_span = 10)

print (len(v_dict_convolved))
v_array_convolved = np.array(list(v_dict_convolved.values()))

#scaler = StandardScaler()
#standardized_vectors = scaler.fit_transform(v_array2)




In [ ]:
#secondary clustering
dimensions = [3,4,8,10]
hdbscan_refined = DBSCAN(min_samples = 45,
                             metric = "euclidean",
                             eps= 0.08,
                             n_jobs=-1)

labels = hdbscan_refined.fit_predict(v_array_convolved[:,dimensions])

core_type_dictionary.update({x: 1 if labels[i] == 1 else -1 for i,x in enumerate(v_dict_convolved.keys())  })

In [ ]:
fig = plt.figure(figsize=(60,3), dpi = 150)

gs = fig.add_gridspec(2, 1,)

x_values = [x for i, x in enumerate(v_dict.keys()) if i in semi_defined_indices]


colors = [f"C{label}"  if label >= 0 else "black"  for label in labels]

bar_plot = fig.add_subplot(gs[1,0])


legend_labels = np.unique(labels)
legend_colors = [f"C{label}"  if label >= 0 else "black" for label in legend_labels]
legend_handles = [mpatches.Patch(color=color, label=label) for label, color in zip(legend_labels, legend_colors)]

bar_plot.bar(x_values, height=1.5, width=1.5, color=colors, edgecolor="none")

bar_plot.legend(handles=legend_handles, title="Core Type", bbox_to_anchor=(1,1), loc="upper left")

bar_plot.set_title("Core Type by X-position")
bar_plot.set_xlabel("X-Value")
bar_plot.set_yticks([])
bar_plot.set_xticks(list(range(0, 31001, 1000)))
bar_plot.set_xticklabels(list(range(0, 31001, 1000)), rotation=70)

ax = fig.add_subplot(gs[0,0])



x_vals = [x for x in v_dict_convolved.keys()]
y_vals = [v_dict_convolved[x][4] for x in v_dict_convolved.keys()]

ax.scatter(x_vals, y_vals, color=colors, s=1)
ax.set_xticks(list(range(0, 31001, 1000)))
ax.set_xticklabels(list(range(0, 31001, 1000)), rotation=70)

plt.show()

In [ ]:
vectors = np.array(list(v_dict_convolved.values()))
x = vectors[:,3]
y = vectors[:,10]
z = vectors[:,8]


fig = go.Figure(data=[go.Scatter3d(
    x=x, y=y, z=z,
    mode='markers',
    marker=dict(size=1.3, color=labels, opacity=1),
)])

fig.update_layout(
    title="3D Scatter of Vectors",
    scene=dict(
        xaxis_title="mean",
        yaxis_title="skew",
        zaxis_title="azimuthal angle",
        camera=dict(projection=dict(type="orthographic"))
    ),
    width=1000,
    height=1000
)

fig.show()

In [ ]:

fig = plt.figure(figsize=(150,3), dpi = 500)

core_types = list(core_type_dictionary.values())
x_positions = list(core_type_dictionary.keys())

colors2 = [f"C{type}" if type >= 0  else (1,1,0,1)  for type in core_types]

ax = fig.add_subplot(111)


ax.scatter( point_cloud[:, 0], point_cloud[:, 1], c=point_cloud[:, 2], cmap='binary', s=0.09, alpha = 0.8)
ax.bar(x_positions, height=20, width= 0.75, color=colors2, edgecolor="none", bottom = -10, alpha = 0.45)



legend_labels2 = np.unique(core_types)


legend_colors2 = [f"C{label}"  if label >= 0  else (1,1,0,1) for label in legend_labels2 ]
legend_handles2 = [mpatches.Patch(color=color, label = (
                                    "half" if label == 0 else 
                                    "empty" if label == 2 else 
                                    "full" if label == 1 else 
                                    label 
                                    )) for label, color in zip(legend_labels2, legend_colors2)]

ax.set_xticks(list(range(0, 31001, 500)))
ax.set_xticklabels(list(range(0, 31001, 500)), rotation=70)
ax.legend(handles=legend_handles2, title="Core Type", bbox_to_anchor=(1,1), loc="upper left")

plt.show()

In [ ]:
#window_size = 7.5
#
#core_type_dictionary_denoised = {}
#
#for x in core_type_dictionary.keys():
#    neighbours = np.unique(point_cloud[point_tree.query_ball_point((x,0), window_size),0])
#    try:
#         counts = Counter(core_type_dictionary[xi] for xi in neighbours)
#    except Exception:
#         print("bad index")
#         pass
#    core_type_dictionary_denoised[x] = counts.most_common(1)[0][0]
#
#fig = plt.figure(figsize=(150,3), dpi = 500)
#
#
#
#
#
#center_y = point_cloud[np.abs(point_cloud[:, 1]).argmin(),1]
#
#mask_y = point_cloud_edges[:, 1] == center_y
#
#
#
#indices = np.where(mask_y)[0] 
#indices = sorted(indices, key = lambda i: point_cloud_edges[i,0])
#
#
#edge_indices = np.where(point_cloud_edges[indices,2] == 1)[0]
#
#
#
#
#for i_start, i_stop in zip(edge_indices, edge_indices[1:]):
#   
#    x_start = point_cloud_edges[indices[i_start],0]
#    x_stop = point_cloud_edges[indices[i_stop],0]
#
#   
#    
#    if ((x_stop - x_start) >= 10):
#        keys = point_cloud_edges[indices[i_start:i_stop],0]
#        values = [core_type_dictionary_denoised[key] for key in keys]
#        mode_value = mode(values)[0]
#        for key in keys:
#            core_type_dictionary_denoised[key] = mode_value
#    else:
#        for key in keys:
#            core_type_dictionary_denoised[key] = -1
#        
#
#
#
#core_types = list(core_type_dictionary_denoised.values())
#x_positions = list(core_type_dictionary_denoised.keys())
#
#colors2 = [f"C{type}" if type >= 0  else (1,1,0,1)  for type in core_types]
#
#ax = fig.add_subplot(111)
#
#
#ax.scatter( point_cloud[:, 0], point_cloud[:, 1], c=point_cloud[:, 2], cmap='binary', s=0.09, alpha = 0.8)
#ax.bar(x_positions, height=20, width= 1, color=colors2, edgecolor="none", bottom = -10, alpha = 0.45)
#
#
#
#legend_labels2 = np.unique(core_types)
#
#
#legend_colors2 = [f"C{label}"  if label >= 0  else (1,1,0,1) for label in legend_labels2 ]
#legend_handles2 = [mpatches.Patch(color=color, label = (
#                                    "half" if label == 0 else 
#                                    "empty" if label == 2 else 
#                                    "full" if label == 4 else 
#                                    label 
#                                    )) for label, color in zip(legend_labels2, legend_colors2)]
#
#ax.set_xticks(list(range(0, 31001, 500)))
#ax.set_xticklabels(list(range(0, 31001, 500)), rotation=70)
#ax.legend(handles=legend_handles2, title="Core Type", bbox_to_anchor=(1,1), loc="upper left")
#
#plt.show()

In [ ]:
core_type_windows = core_type_dictionary.copy()

center_y = point_cloud[np.abs(point_cloud[:, 1]).argmin(),1]

mask_y = point_cloud_edges[:, 1] == center_y



indices = np.where(mask_y)[0] 
indices = sorted(indices, key = lambda i: point_cloud_edges[i,0])

edge_indices = np.where(point_cloud_edges[indices,2] == 1)[0]

x_values = point_cloud_edges[indices,0]
x_values = x_values[edge_indices]

window_dict = {}

rubble_windows = []


for i_start, i_stop in zip(edge_indices, edge_indices[1:]):
      
    x_start = point_cloud_edges[indices[i_start],0]
    x_stop = point_cloud_edges[indices[i_stop],0]

    keys = point_cloud_edges[indices[i_start + 1:i_stop],0]

    if ((x_stop - x_start) >= 10):
        values = [core_type_windows[key] for key in keys]
        mode_value = mode(values)[0]
        window_dict[int(i_start)] = int(mode_value)
    else:
        window_dict[int(i_start)] = -1


keys_to_remove = []

for index1, index2 in zip(list(window_dict.keys()), list(window_dict.keys())[1:]):
    type1 = window_dict[index1]
    type2 = window_dict[index2]

    if type1 == -1 and type2 == -1:
        keys_to_remove.append(index2)

for key in keys_to_remove:
    del window_dict[key]

for (i_start,i_stop) in zip(list(window_dict.keys()), list(window_dict.keys())[1:]):

    core_type = window_dict[i_start]

    keys = point_cloud_edges[indices[i_start + 1:i_stop],0]

    if core_type == -1:
        x_start = point_cloud_edges[indices[i_start],0]
        x_stop = point_cloud_edges[indices[i_stop],0]
        rubble_windows.append((x_start, x_stop)) 


    for key in keys:
        if key not in x_values:
            core_type_windows[key] = core_type


fig = plt.figure(figsize=(200,3), dpi = 300)


gs = fig.add_gridspec(1, 1, hspace = 0.8)

downsampled_indices = np.random.choice(point_cloud.shape[0], size=90000, replace=False)

x_downsampled = point_cloud[:, 0]
y_downsampled = point_cloud[:, 1]
z_downsampled = point_cloud[:, 2]


ax_scatter = fig.add_subplot(gs[0,0])
ax_scatter.scatter(
x_downsampled,
y_downsampled,
c=z_downsampled,
cmap='binary_r',
s = 0.25
)

core_types = list(core_type_windows.values())
x_positions = list(core_type_windows.keys())




ax_scatter.set_title('LIDAR data heatmap')
ax_scatter.set_xlabel('X-axis')
ax_scatter.set_ylabel('Y-axis')



legend_labels = np.unique(core_types)
legend_colors = [f"C{label}"  if label >= 0 else (0,1,1,1) for label in core_types]
legend_handles = [mpatches.Patch(color=color, label=label) for label, color in zip(legend_labels, legend_colors)]

ax_scatter.bar(x_positions, height=20, width=1, color=legend_colors, edgecolor="none", bottom = -10, alpha = 0.4)

ax_scatter.legend(handles=legend_handles, title="Core Type", bbox_to_anchor=(1,1), loc="upper left")

plt.show()




In [ ]:
print(x_positions)

In [ ]:
print(rubble_windows)

In [ ]:
#h_dict = get_hessian_dictionary(point_cloud)
#
#
#
#eigens = h_dict.values()
#
#scaled_eigenvectors = []
#
#for eigenvalues, eigenvectors in eigens:
#    scaled_vectors = eigenvectors * eigenvalues[:, np.newaxis]  
#    scaled_eigenvectors.append(scaled_vectors)
#

In [ ]:
#x = np.concatenate([scaled_vec[0] for scaled_vec in scaled_eigenvectors])
#y = np.concatenate([scaled_vec[1] for scaled_vec in scaled_eigenvectors])
#
#fig = go.Figure(data=[go.Scatter3d(
#    x=x, y=y,
#    mode='markers',
#    marker=dict(size=1.3, opacity=1),
#)])
#
#fig.update_layout(
#    title="3D Scatter of Vectors by Core Type",
#    scene=dict(
#        xaxis_title="0",
#        yaxis_title="1",
#        zaxis_title="2",
#        camera=dict(projection=dict(type="orthographic"))
#    ),
#    width=1000,
#    height=1000
#)
#
#fig.show()

In [ ]:
# Define the directory where the LIDAR scans are located
# lidar_scan_dir = r"C:\Users\mcastillo\gitlocal\xray_spot\data\test_new_setup\calibration_xray\Core_000_Box_000_of_000_Pass_007"
# lidar_scan_dir = r"\\192.168.1.100\CoreScan3-2\Acquisitions\RnD\XRF_SPOT\AI21\xray2lidar\Core_000_Box_000_of_000_Pass_019"
lidar_scan_dir = r"\\192.168.1.100\CoreScan3-2\Acquisitions\Trailer-Calibration-Scans-DO-NOT-MOVE-OR-DELETE\AI21\Macassa3.5\AI21-Dec27-2024\Core_000_Box_005_of_999_Part_5_of_6"
 
# Load the point cloud from the LIDAR scan
lidar_filename, point_cloud, intensity_map = load_point_cloud(lidar_scan_dir)
 
# Create a scatter plot of the point cloud
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax = display_scatter(point_cloud, ax=ax)
 
# Read the configuration for the LIDAR scan
conf = read_component_conf(scan_dir=lidar_scan_dir)
 
# Get the starting Y coordinate for the LIDAR scan
y_offset = float(conf['LIDAR_CAMERA']['Acquisition']['Y.Start'])
print(f"Y offset {y_offset}")
 
# Render the LIDAR scan to an image
Tr_lidar2render, lidar_render, inpaint_mask, lidar_render_inpaint, (x_min, y_min, z_min), (x_max, y_max, z_max), reset_mask = render_lidar(point_cloud, flip_x = True, flip_y=True, inpaint_size = 10, clean_nan=True)
 
# Render the intensity map image from the LIDAR scan
intensity_img = render_lidar_intensity_map(point_cloud, intensity_map, Tr_lidar2render, (lidar_render.shape[1], lidar_render.shape[0]))
 
# Display the rendered LIDAR scan and the intensity map image
fig, axes = plt.subplots(1, 2, figsize=(10, 10))
 
# Display the rendered LIDAR scan in the first subplot
axes[0].imshow(lidar_render_inpaint)
 
# Display the intensity map image in the second subplot
axes[1].imshow(intensity_img)
 
# Print lidar2render
print(Tr_lidar2render)
 